In [68]:
import requests
import json
import csv
from pprint import pprint
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

In [96]:
## get the chemicals by route from our cdc exposure data
def get_route_object(route, data):
    try:
        route_obj = data[route]
        route_obj.update({'route': route})
        return route_obj
    except Exception as e:
        print(e)
        return None

with open('cdc_exposure_routes.json', 'r') as routes:
    routes = json.load(routes)
    
## get chebi 2 pathway map
# curl -O "https://reactome.org/download/current/ChEBI2Reactome.txt"

# Parse chebi pathways tab sep file from reactome into json
chebi_pathways = []
with open('ChEBI2Reactome.txt', 'r') as cheb2path:
    cheb2path = csv.reader(cheb2path, delimiter='\t')
    for index, line in enumerate(cheb2path):
        chebi_pathways.append({
            'chebi': line[0],
            'reactome': line[1],
            'pathway': line[3],
            'evidence': line[4],
            'species': line[5]
            
        })

In [112]:
## generate a pandas dataframe for all chems with given route
# route_chems = get_route_object(route='Volatile Organic Compounds (VOC)', data=routes)
route_chems = get_route_object(route='Metals and Metalloids', data=routes)

seed_data = []
for route_chem in route_chems['chemicals']:
    seed_data.append({
        'route': route_chems['route'],
        'chebi': route_chem['chebi'],
        'chemical': route_chem['label'],
    }
    )
chem_datframe = pd.DataFrame(data=seed_data)

In [113]:
chem_datframe

chebi               chemical                  route
0   30513               antimony  Metals and Metalloids
1   27563                arsenic  Metals and Metalloids
2   82392          arsenobetaine  Metals and Metalloids
3   49900          arsenous acid  Metals and Metalloids
4   27131  trimethylarsine oxide  Metals and Metalloids
5   30514                caesium  Metals and Metalloids
6   29036                 copper  Metals and Metalloids
7   27889                   lead  Metals and Metalloids
8   27568               selenium  Metals and Metalloids
9   30440               thallium  Metals and Metalloids
10  27998               tungsten  Metals and Metalloids
11  32996                uranium  Metals and Metalloids
12  30185                   zinc  Metals and Metalloids

In [114]:
## get all pathways that the chems are involved in into new dataframe
pathways_list = []
for index, row in chem_datframe.iterrows():
    chebi = row[0]
    chemical = row[1]
    route = row[2]
    for path in chebi_pathways:
        if chebi == path['chebi']:
            path.update({
                'chemical': chemical,
                'route': route,
            })
            pathways_list.append(path)
all_dataframe = pd.DataFrame(data=pathways_list)  
all_dataframe.head()

chebi chemical evidence                                    pathway  \
0  29036   copper      IEA  Detoxification of Reactive Oxygen Species   
1  29036   copper      IEA            Ion transport by P-type ATPases   
2  29036   copper      IEA                    Elastic fibre formation   
3  29036   copper      IEA           Crosslinking of collagen fibrils   
4  29036   copper      IEA  Detoxification of Reactive Oxygen Species   

        reactome                  route               species  
0  R-ATH-3299685  Metals and Metalloids  Arabidopsis thaliana  
1   R-ATH-936837  Metals and Metalloids  Arabidopsis thaliana  
2  R-BTA-1566948  Metals and Metalloids            Bos taurus  
3  R-BTA-2243919  Metals and Metalloids            Bos taurus  
4  R-BTA-3299685  Metals and Metalloids            Bos taurus

In [115]:
# api utility methods
def get_pathway_participants(pathway_id):
    url = "https://reactome.org/ContentService/data/participants/{}/participatingPhysicalEntities".format(pathway_id)
    results = requests.get(url).json()
    return results

def reactome_id2reference_entity(db_id):
    url = 'https://reactome.org/ContentService/data/query/enhanced/{}'.format(db_id)
    results = requests.get(url).json()
    return results['referenceEntity']

def execute_query(query):
    endpoint = SPARQLWrapper('https://query.wikidata.org/sparql')
    endpoint.setQuery(query)
    endpoint.setReturnFormat(JSON)
    return endpoint.query().convert()

def mygene_map_uniprot_entrez(uniprot):
    url ='http://mygene.info/v3/query'
    params = {'q': uniprot,
              'fields': 'entrezgene',
             }
    results = requests.get(url, params).json()
    final = None
    try:
        if results['total'] == 1:
            final = str(results['hits'][0]['entrezgene'])
    except Exception as e:
        pass
    return final

def get_monarch_diseases(entrez):
    entrez = 'NCBIGene:{}'.format(entrez)
    url = 'https://api-dev.monarchinitiative.org/api/bioentity/gene/{}/diseases/'.format(entrez)
    results = requests.get(url).json()
    diseases = []
    if len(results['associations']) > 0:
        for assoc in results['associations']:
            diseases.append({
                'mondo': assoc['object']['id'], 
                'disease': assoc['object']['label']
            })
    return diseases

In [118]:
# gets pathway participants (proteins) from reactome and makes new combined dataframe
participant_uniprots = []
for index, row in all_dataframe.iterrows():
    participants = get_pathway_participants(pathway_id=row[4])
    for part in participants:
        new_doc = {
            'chebi': row[0],
            'chemical': row[1],
            'evidence': row[2],
            'pathway': row[3],
            'reactome': row[4],
            'route': row[5],
            'species': row[6]
        }

        if part['className']  ==  'Protein':       
            part_ref = reactome_id2reference_entity(part['dbId'])
            if 'UniProt' == part_ref['databaseName']:
                new_doc.update({'uniprot': part_ref['identifier'] })
                participant_uniprots.append(new_doc)
all_data_with_proteins = pd.DataFrame(data=participant_uniprots)
all_data_with_proteins.head()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108


chebi chemical evidence                                    pathway  \
0    29036   copper      IEA            Ion transport by P-type ATPases   
1    29036   copper      IEA                    Elastic fibre formation   
2    29036   copper      IEA                    Elastic fibre formation   
3    29036   copper      IEA                    Elastic fibre formation   
4    29036   copper      IEA  Detoxification of Reactive Oxygen Species   
5    29036   copper      IEA  Detoxification of Reactive Oxygen Species   
6    29036   copper      IEA  Detoxification of Reactive Oxygen Species   
7    29036   copper      IEA  Detoxification of Reactive Oxygen Species   
8    29036   copper      IEA  Detoxification of Reactive Oxygen Species   
9    29036   copper      IEA  Detoxification of Reactive Oxygen Species   
10   29036   copper      IEA                 Metal ion SLC transporters   
11   29036   copper      IEA                 Metal ion SLC transporters   
12   29036   copper      IEA                 Metal ion SLC transporters   
13   29036   copper      IEA                 Metal ion SLC transporters   
14   29036   copper      IEA                 Metal ion SLC transporters   
15   29036   copper      IEA                 Metal ion SLC transporters   
16   29036   copper      IEA                 Metal ion SLC transporters   
17   29036   copper      IEA               Metallothioneins bind metals   
18   29036   copper      IEA               Metallothioneins bind metals   
19   29036   copper      IEA                       Melanin biosynthesis   
20   29036   copper      IEA                       Melanin biosynthesis   
21   29036   copper      IEA                       Melanin biosynthesis   
22   29036   copper      IEA                  Iron uptake and transport   
23   29036   copper      IEA                  Iron uptake and transport   
24   29036   copper      IEA                  Iron uptake and transport   
25   29036   copper      IEA                  Iron uptake and transport   
26   29036   copper      IEA                  Iron uptake and transport   
27   29036   copper      IEA                  Iron uptake and transport   
28   29036   copper      IEA                  Iron uptake and transport   
29   29036   copper      IEA                  Iron uptake and transport   
..     ...      ...      ...                                        ...   
400  27889     lead      IEA                          Heme biosynthesis   
401  27889     lead      IEA                          Heme biosynthesis   
402  27889     lead      IEA                          Heme biosynthesis   
403  27889     lead      IEA                          Heme biosynthesis   
404  27889     lead      IEA                          Heme biosynthesis   
405  27889     lead      IEA                          Heme biosynthesis   
406  27889     lead      IEA                          Heme biosynthesis   
407  27889     lead      TAS                          Heme biosynthesis   
408  27889     lead      TAS                          Heme biosynthesis   
409  27889     lead      TAS                          Heme biosynthesis   
410  27889     lead      IEA                          Heme biosynthesis   
411  27889     lead      IEA                          Heme biosynthesis   
412  27889     lead      IEA                          Heme biosynthesis   
413  27889     lead      IEA                          Heme biosynthesis   
414  27889     lead      IEA                          Heme biosynthesis   
415  27889     lead      IEA                          Heme biosynthesis   
416  27889     lead      IEA                          Heme biosynthesis   
417  27889     lead      IEA                          Heme biosynthesis   
418  27889     lead      IEA                          Heme biosynthesis   
419  27889     lead      IEA                          Heme biosynthesis   
420  27889     lead      IEA                          Heme biosynthesis   
421  27889     lead      IEA  

In [123]:
# map to ncbi gene 
all_data_with_proteins['ncbi_gene'] = all_data_with_proteins['uniprot'].apply(mygene_map_uniprot_entrez)

In [126]:
# drop the rows with no gene id
all_data_with_proteins = all_data_with_proteins[all_data_with_proteins.ncbi_gene.notnull()]

In [127]:
all_data_with_proteins.head()

chebi chemical evidence                                    pathway  \
0  29036   copper      IEA            Ion transport by P-type ATPases   
2  29036   copper      IEA                    Elastic fibre formation   
4  29036   copper      IEA  Detoxification of Reactive Oxygen Species   
5  29036   copper      IEA  Detoxification of Reactive Oxygen Species   
8  29036   copper      IEA  Detoxification of Reactive Oxygen Species   

        reactome                  route               species uniprot  \
0   R-ATH-936837  Metals and Metalloids  Arabidopsis thaliana  Q9LT02   
2  R-BTA-1566948  Metals and Metalloids            Bos taurus  Q2KJ89   
4  R-BTA-3299685  Metals and Metalloids            Bos taurus  Q95108   
5  R-BTA-3299685  Metals and Metalloids            Bos taurus  Q95108   
8  R-BTA-3299685  Metals and Metalloids            Bos taurus  Q9BGI1   

  ncbi_gene  
0    832428  
2    535185  
4    281557  
5    281557  
8    282885

In [128]:
#get all diseases from monarchinitiative by gene id
gene2disease = []
genes = all_data_with_proteins['ncbi_gene'].tolist()
for index, gene in enumerate(genes):
    diseases = get_monarch_diseases(gene)
    for disease in diseases:
        disease.update({'gene': gene})
        gene2disease.append(disease)
gene2disease_df = pd.DataFrame(data=gene2disease)
gene2disease_df.head()

disease    gene             mondo
0                   Dun coat colour in Dexter cattle  282105  OMIA:001249-9913
1                          Coat colour, brown in dog  403479  OMIA:001249-9615
2  cutis laxa with severe pulmonary, gastrointest...    8425     MONDO:0013170
3                        Duchenne muscular dystrophy    8425     MONDO:0010679
4                   cutis laxa, autosomal dominant 1   10516     MONDO:0007411

In [129]:
# merge gene2disease with all data for final ouput
merged_final = pd.merge(all_data_with_proteins, gene2disease_df, how='inner', left_on='ncbi_gene', right_on='gene')

In [130]:
merged_final

chebi chemical evidence                          pathway       reactome  \
0    29036   copper      IEA             Melanin biosynthesis  R-BTA-5662702   
1    29036   copper      IEA             Melanin biosynthesis  R-CFA-5662702   
2    29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
3    29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
4    29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
5    29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
6    29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
7    29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
8    29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
9    29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
10   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
11   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
12   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
13   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
14   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
15   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
16   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
17   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
18   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
19   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
20   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
21   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
22   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
23   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
24   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
25   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
26   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
27   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
28   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
29   29036   copper      TAS          Elastic fibre formation  R-HSA-1566948   
..     ...      ...      ...                              ...            ...   
322  29036   copper      TAS        Iron uptake and transport   R-HSA-917937   
323  29036   copper      TAS        Iron uptake and transport   R-HSA-917937   
324  29036   copper      TAS        Iron uptake and transport   R-HSA-917937   
325  29036   copper      TAS        Iron uptake and transport   R-HSA-917937   
326  29036   copper      TAS        Iron uptake and transport   R-HSA-917937   
327  29036   copper      TAS        Iron uptake and transport   R-HSA-917937   
328  29036   copper      TAS  Ion transport by P-type ATPases   R-HSA-936837   
329  29036   copper      TAS  Ion transport by P-type ATPases   R-HSA-936837   
330  29036   copper      TAS  Ion transport by P-type ATPases   R-HSA-936837   
331  29036   copper      TAS  Ion transport by P-type ATPases   R-HSA-936837   
332  29036   copper      TAS  Ion transport by P-type ATPases   R-HSA-936837   
333  29036   copper      TAS  Ion transport by P-type ATPases   R-HSA-936837   
334  29036   copper      TAS  Ion transport by P-type ATPases   R-HSA-936837   
335  29036   copper      TAS  Ion transport by P-type ATPases   R-HSA-936837   
336  29036   copper      TAS  Ion transport by P-type ATPases   R-HSA-936837   
337  29036   copper      TAS  Ion transport by P-type ATPases   R-HSA-936837   
338  29036   copper      TAS  Ion transport by P-type ATPases   R-HSA-936837   
339  29036   copper      TAS  Ion transport by P-type ATPases   R-HSA-936837   
340  